## Approach using YOLO inference and Keras-OCR framework to detect texts (names) on Mapillary images

**Aims**:
* Use the best trained model from YOLOv12 to detect text areas in the street-level images (SLI)
* Follow up with Keras-OCR framework to text (toponyms) recognition from these areas

#### Import necessary libraries and modules

In [2]:
import os
import numpy as np
import gc
import torch
from datetime import datetime
import pandas as pd
import geopandas as gpd
import fiona
from tqdm.notebook import tqdm # tqdm no notebook

In [4]:
# recarrega automaticamente as funções
%load_ext autoreload
%autoreload 2

In [5]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [7]:
# Check the current working directory
os.getcwd()

'c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms'

In [6]:
# Import function from the modules train_yolov12
from src.txt_detect_Yv12_keras_ocr import init_recognizer, yolo_predict_and_crop, run_ocr_on_crops, rebuild_df_crops

#### Applying YOLOv12 best trained model to predict and crop areas with text in the SLI

In [7]:
# Path to the best model
BEST_MAP = os.path.join('yolov12','runs', 'mapillary_only_v1', 'weights', 'best.pt')
print(BEST_MAP)
assert os.path.exists(BEST_MAP), f"Pesos não encontrados: {BEST_MAP}"

yolov12\runs\mapillary_only_v1\weights\best.pt


In [8]:
# === 1. Predição YOLO + crops ===
base = os.getcwd()
source_dir = os.path.join(base, "results", "5_CLIP_download_img_mpl", "20250825")
output_dir = os.path.join(base, "results", "6_YOLO_KerasOCR_results", "mpl")
os.makedirs(output_dir, exist_ok=True)

df_crops = yolo_predict_and_crop(
    weights=BEST_MAP,
    source_dir=source_dir,
    output_dir=output_dir ,
    img_size=640,
    conf_thres=0.25,
    iou_thres=0.45,
    device="cuda" if torch.cuda.is_available() else "cpu",
    workers=4
)
print("Total crops:", len(df_crops))

Processando edif_comerc_serv\healthcare_any: 100%|██████████| 98/98 [00:09<00:00,  9.81img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:09<00:00, 10.69img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:12<00:00,  8.08img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:11<00:00,  8.90img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:10<00:00,  9.13img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:09<00:00, 10.10img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:12<00:00,  7.79img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:08<00:00, 12.12img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:10<00:00,  9.82img/s]
Processando edif_constr_lazer\amenity_arts_centre: 100%|██████████| 100/100 [00:09<00:00, 

[INFO] Log de imagens sem detecção salvo em c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6_YOLO_KerasOCR_results\mpl\no_detections.csv
Total crops: 20029


In [ ]:
# Recuperar o df_crops com mapeamento das saidas
base = os.getcwd()
output_dir = os.path.join(base, "results", "6_YOLO_KerasOCR_results","mpl")
df_crops = rebuild_df_crops(output_dir)

print("Total crops encontrados:", len(df_crops))
df_crops.head()

Total crops encontrados: 182


,osmId_formatado,classe,tag_value,image_id,path_crop
0,relation_5462805,edif_constr_lazer,leisure_sports_centre,1447298768958676,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\0...
1,relation_5462805,edif_constr_lazer,leisure_sports_centre,175062311211008,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\0...
2,relation_5462805,edif_constr_lazer,leisure_sports_centre,182625110412851,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\0...
3,relation_5462805,edif_constr_lazer,leisure_sports_centre,4025966667458518,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\0...
4,relation_5462805,edif_constr_lazer,leisure_sports_centre,953542098792222,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\0...


#### Applying Keras-OCR to extract the texts from the predicted regions by YOLOv12

In [ ]:
# Inicializa OCR Recognizer
recognizer = init_recognizer()

Looking for C:\Users\SIGEO\.keras-ocr\crnn_kurapan.h5


In [11]:
# === 2. OCR nos crops ===
df_results = run_ocr_on_crops(df_crops, recognizer)

1/1 [==============================] - 0s 59ms/step


OCR nos crops:  93%|█████████▎| 18661/20029 [1:17:53<05:49,  3.92img/s]

1/1 [==============================] - 0s 22ms/step


OCR nos crops:  93%|█████████▎| 18662/20029 [1:17:54<05:34,  4.09img/s]

1/1 [==============================] - 0s 22ms/step


...

OCR nos crops: 100%|█████████▉| 20027/20029 [1:23:19<00:00,  4.03img/s]

1/1 [==============================] - 0s 53ms/step


OCR nos crops: 100%|█████████▉| 20028/20029 [1:23:19<00:00,  4.00img/s]

1/1 [==============================] - 0s 45ms/step


OCR nos crops: 100%|██████████| 20029/20029 [1:23:20<00:00,  4.01img/s]


In [12]:
# Visualizar
display(df_results)

,osmId_formatado,classe,tag_value,image_id,words_sli,texto_sli,path_crop
0,relation_8013500,cbge_area_verde,leisure_park,384344719496346,[],,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
1,relation_8013500,cbge_area_verde,leisure_park,801899957379381,a,a,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
2,way_100290466,cbge_area_verde,leisure_park,520526909141952,le,l e,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
3,way_1026854836,cbge_area_verde,leisure_park,149091137259541,ies,i e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
4,way_1026854836,cbge_area_verde,leisure_park,1605443426327906,baa,b a a,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
...,...,...,...,...,...,...,...
20024,node_5363657442,edif_turistica,tourism_viewpoint,953465825485344,niee,n i e e,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
20025,node_5363657452,edif_turistica,tourism_viewpoint,1143601826125378,se,s e,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
20026,node_5363657452,edif_turistica,tourism_viewpoint,1183758152100706,des,d e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
20027,node_5363657452,edif_turistica,tourism_viewpoint,223260119237378,is,i s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...


In [13]:
# Salvar
df_results.to_csv(os.path.join(output_dir,"mlp_ocr_results.csv"), index=False)

#### Retriving and associate df_results to mapillary points (clip_vitL14_mpl_matches)

In [8]:
# Recuperar o df_results a partir do csv salvo
base = os.getcwd()
csv_path = os.path.join(base, "results", "6_YOLO_KerasOCR_results","mpl", "mlp_ocr_results.csv")

df_results = pd.read_csv(csv_path)

# Conferir
print(df_results.shape)
df_results.head()

(20029, 7)


,osmId_formatado,classe,tag_value,image_id,words_sli,texto_sli,path_crop
0,relation_8013500,cbge_area_verde,leisure_park,384344719496346,[],NaN,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
1,relation_8013500,cbge_area_verde,leisure_park,801899957379381,a,a,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
2,way_100290466,cbge_area_verde,leisure_park,520526909141952,le,l e,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
3,way_1026854836,cbge_area_verde,leisure_park,149091137259541,ies,i e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
4,way_1026854836,cbge_area_verde,leisure_park,1605443426327906,baa,b a a,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...


In [ ]:
# Versao mais robusta
base = os.getcwd()
csv_path = os.path.join(base, "results", "6_YOLO_KerasOCR_results", "mpl", "mlp_ocr_results.csv")

# total de linhas (# -1 cabeçalho)
total_lines = sum(1 for _ in open(csv_path, encoding="utf-8")) - 1
chunk_size = 50_000

dfs = []
for chunk in tqdm(pd.read_csv(csv_path, chunksize=chunk_size), 
                  total=(total_lines // chunk_size) + 1, 
                  desc="Carregando CSV"):
    dfs.append(chunk)

df_results = pd.concat(dfs, ignore_index=True)

print("Shape:", df_results.shape)
display(df_results.head())

Carregando CSV:   0%|          | 0/1 [00:00<?, ?it/s]

Shape: (20029, 7)


,osmId_formatado,classe,tag_value,image_id,words_sli,texto_sli,path_crop
0,relation_8013500,cbge_area_verde,leisure_park,384344719496346,[],NaN,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
1,relation_8013500,cbge_area_verde,leisure_park,801899957379381,a,a,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
2,way_100290466,cbge_area_verde,leisure_park,520526909141952,le,l e,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
3,way_1026854836,cbge_area_verde,leisure_park,149091137259541,ies,i e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
4,way_1026854836,cbge_area_verde,leisure_park,1605443426327906,baa,b a a,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...


In [ ]:
# Reload do clip_vitL14_mpl_matches do gpkg "20250822_152831_clip_osm_mpl_results_final.gpkg"

# Caminhos
base = os.getcwd()
results_path = "results/4_CLIP_results/2_ViT_L14/20250819"
fp_gpkg = os.path.join(base, results_path, "20250822_152831_clip_osm_mpl_results_final.gpkg")
layer_name = "clip_vitL14_mpl_matches"

# Verifica se a layer existe
layers = fiona.listlayers(fp_gpkg)
if layer_name not in layers:
    raise ValueError(f"Layer '{layer_name}' não encontrada no arquivo. Layers disponíveis: {layers}")

# Conta feições da layer
with fiona.open(fp_gpkg, layer=layer_name) as src:
    total_features = len(src)
print(f"Total de feições na layer '{layer_name}': {total_features}")

# Leitura em chunks
chunk_size = 10_000
dfs = []

with fiona.open(fp_gpkg, layer=layer_name) as src:
    features = []
    for i, feat in enumerate(tqdm(src, total=total_features, desc=f"Carregando '{layer_name}'")):
        features.append(feat)
        if (i + 1) % chunk_size == 0 or (i + 1) == total_features:
            # CRS: Fiona pode retornar dict (src.crs) ou WKT (src.crs_wkt)
            crs = src.crs
            if crs is None and getattr(src, "crs_wkt", None):
                try:
                    from pyproj import CRS
                    crs = CRS.from_wkt(src.crs_wkt).to_wkt()
                except Exception:
                    crs = None

            df_chunk = gpd.GeoDataFrame.from_features(features, crs=crs)
            dfs.append(df_chunk)

            # Limpeza
            features.clear()
            del df_chunk
            gc.collect()

# Concatena tudo em um único GeoDataFrame final
if not dfs:
    # vazio
    gdf_mpl_final = gpd.GeoDataFrame(geometry=[])
else:
    gdf_mpl_final = gpd.GeoDataFrame(
        pd.concat(dfs, ignore_index=True),
        crs=dfs[0].crs
    )

print("GeoDataFrame final:", gdf_mpl_final.shape)

# Libera memória intermediária
del dfs
gc.collect()

Total de feições na layer 'clip_vitL14_mpl_matches': 108475


Carregando 'clip_vitL14_mpl_matches':   0%|          | 0/108475 [00:00<?, ?it/s]

GeoDataFrame final: (108475, 239)


18

In [10]:
# Preparar os IDs para join
df_results["image_id"] = df_results["image_id"].astype(str)
gdf_mpl_final["image_id"] = gdf_mpl_final["image_id"].astype(str)

In [ ]:
# Join OCR → CLIP (gdf_mpl_final → clip_vitL14_mpl_matches)

print("Iniciando merge OCR → CLIP match results em chunks ...")
chunk_size = 20000
dfs_merged = []

for i in tqdm(np.arange(0, len(df_results), chunk_size), desc="Join OCR-CLIP", unit="chunk"):
    df_chunk = df_results.iloc[i:i+chunk_size]
    merged_chunk = gdf_mpl_final.reset_index().merge(
        df_chunk.reset_index(),
        on="image_id",
        how="left",
        copy=False
    )
    dfs_merged.append(merged_chunk)

gdf_merged = pd.concat(dfs_merged, ignore_index=True)
print("Merge concluído:", gdf_merged.shape)
display(gdf_merged.head())

Iniciando merge OCR → CLIP em chunks ...


Join OCR-CLIP:   0%|          | 0/2 [00:00<?, ?chunk/s]

Merge concluído: (229006, 247)


,index_x,geometry,id_celula,classe_x,tag,value,@changesetId,@contributionChangesetId,@creation,@osmId,...,dist_m,score,unique_key,index_y,osmId_formatado,classe_y,tag_value,words_sli,texto_sli,path_crop
0,0,POINT (-44.04466 -19.99024),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,37.792386,0.355469,node/1830091373_294025759131160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,POINT (-44.04468 -19.99028),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,37.804249,0.584961,node/1830091373_307923321060745,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,POINT (-44.04470 -19.99031),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,38.413131,0.592773,node/1830091373_982441749168567,11289.0,node_1830091373,edif_ensino,amenity_school,ses,s e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...
3,3,POINT (-44.04463 -19.99021),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,38.629732,0.440186,node/1830091373_798420737483679,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,POINT (-44.04473 -19.99034),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,39.079634,0.346680,node/1830091373_970234847060373,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Reload dos POIs OSM (com classes atualizadas)
# === Caminho do step7 ===
base = os.getcwd()
results_path = "results/2_toponyms_retrieval/step7_latest_name_ohsome"
gpkg_path = os.path.join(base, results_path, "step7_consolidado_ohsome_filtrado.gpkg")
layer_step7 = "step7_consolidado_ohsome_filtrado"

# Verifica layers disponíveis
layers = fiona.listlayers(gpkg_path)
if layer_step7 not in layers:
    raise ValueError(f"Layer '{layer_step7}' não encontrada. Disponíveis: {layers}")

# Conta feições
with fiona.open(gpkg_path, layer=layer_step7) as src:
    total_features = len(src)
print(f"Total de feições em step7: {total_features}")

# === Leitura em chunks ===
chunk_size = 20_000
dfs = []

with fiona.open(gpkg_path, layer=layer_step7) as src:
    features = []
    for i, feat in enumerate(tqdm(src, total=total_features, desc=f"Carregando {layer_step7}")):
        features.append(feat)
        if (i + 1) % chunk_size == 0 or (i + 1) == total_features:
            crs = src.crs
            if crs is None and getattr(src, "crs_wkt", None):
                from pyproj import CRS
                try:
                    crs = CRS.from_wkt(src.crs_wkt).to_wkt()
                except Exception:
                    crs = None

            df_chunk = gpd.GeoDataFrame.from_features(features, crs=crs)
            dfs.append(df_chunk)

            features.clear()
            del df_chunk
            gc.collect()

# Concatena todos os chunks
gdf_step7 = gpd.GeoDataFrame(
    pd.concat(dfs, ignore_index=True),
    crs=dfs[0].crs if dfs else None
)
del dfs
gc.collect()

print("GeoDataFrame step7:", gdf_step7.shape)
display(gdf_step7.head())


Total de feições em step7: 1627


Carregando step7_consolidado_ohsome_filtrado:   0%|          | 0/1627 [00:00<?, ?it/s]

GeoDataFrame step7: (1627, 190)


,geometry,id_celula,classe,tag,value,@changesetId,@contributionChangesetId,@creation,@osmId,@osmType,...,addr:province,indoor,building:use,training,subject:wikidata,subject:wikipedia,payment:account_cards,shop,int_name,nohousenumber
0,POINT (-44.05274 -19.99391),200ME60348N90864,cbge_praca,leisure,park,12479899,12479899.0,1,way/172871822,way,...,None,None,None,None,None,None,None,None,None,None
1,POINT (-44.05412 -19.99387),200ME60346N90862,edif_constr_lazer,amenity,library,12479899,12479899.0,1,node/1837185237,node,...,None,None,None,None,None,None,None,None,None,None
2,POINT (-44.05395 -19.99415),200ME60346N90864,edif_ensino,amenity,school,56647215,56647215.0,1,way/172871823,way,...,None,None,None,None,None,None,None,None,None,None
3,POINT (-44.06307 -19.97691),200ME60338N90882,cbge_praca,leisure,park,105601725,105601725.0,1,way/511117205,way,...,None,None,None,None,None,None,None,None,None,None
4,POINT (-44.05539 -19.98152),200ME60346N90876,edif_ensino,amenity,school,56647215,56647215.0,1,way/563950983,way,...,None,None,None,None,None,None,None,None,None,None


In [14]:
# === Atualizar classe_x no gdf_merged ===

# Garantir que ambas têm a chave @osmId
if "@osmId" not in gdf_step7.columns:
    raise KeyError("Coluna @osmId não encontrada em step7")
if "@osmId" not in gdf_merged.columns:
    raise KeyError("Coluna @osmId não encontrada em gdf_merged")

# Garante cópias (não mexe nos originais)
gdf_merged_upd = gdf_merged.copy()
df_update = gdf_step7[["@osmId", "classe"]].drop_duplicates().copy()

print("Fazendo merge otimizado para atualizar 'classe' ...")

gdf_merged_upd = gdf_merged_upd.merge(
    df_update,
    on="@osmId",
    how="left",
    suffixes=("_old", "_new")  # força sufixos
)

# Checar colunas após merge
print("Colunas após merge:", gdf_merged_upd.columns.tolist())

# Atualizar se ambas existirem
if "classe_new" in gdf_merged_upd.columns and "classe_old" in gdf_merged_upd.columns:
    gdf_merged_upd["classe"] = gdf_merged_upd["classe_new"].combine_first(
        gdf_merged_upd["classe_old"]
    )
    gdf_merged_upd = gdf_merged_upd.drop(columns=["classe_old", "classe_new"])

elif "classe_new" in gdf_merged_upd.columns:
    # caso só exista new
    gdf_merged_upd = gdf_merged_upd.rename(columns={"classe_new": "classe"})

elif "classe_old" in gdf_merged_upd.columns:
    # caso só exista old
    gdf_merged_upd = gdf_merged_upd.rename(columns={"classe_old": "classe"})

print("Atualização concluída:", gdf_merged_upd.shape)
display (gdf_merged_upd.head())

Fazendo merge otimizado para atualizar 'classe' ...
Colunas após merge: ['index_x', 'geometry', 'id_celula', 'classe_x', 'tag', 'value', '@changesetId', '@contributionChangesetId', '@creation', '@osmId', '@osmType', '@timestamp', '@version', 'addr:housenumber', 'addr:street', 'amenity', 'contact:phone', 'name', 'operator', 'leisure', '@geometryChange', '@tagChange', 'addr:city', 'addr:postcode', 'addr:suburb', 'short_name', 'sport', 'type', 'healthcare', 'healthcare:speciality', 'operator:type', 'phone', 'tourism', 'opening_hours', 'landuse', 'natural', 'wheelchair', 'admin_level', 'source', 'building', 'dog', 'email', 'fixme', 'phone_1', 'alt_name', 'check_date', 'brand', 'brand:wikidata', 'dispensing', 'internet_access', 'internet_access:fee', 'public_transport', 'railway', 'station', 'subway', 'wikidata', 'garden:type', 'theatre:genre', 'website', 'brand:wikipedia', 'surface', 'school', 'emergency', 'government', 'office', 'smoking', 'building:levels', 'height', 'grades', 'religion'

,index_x,geometry,id_celula,classe_x,tag,value,@changesetId,@contributionChangesetId,@creation,@osmId,...,score,unique_key,index_y,osmId_formatado,classe_y,tag_value,words_sli,texto_sli,path_crop,classe
0,0,POINT (-44.04466 -19.99024),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.355469,node/1830091373_294025759131160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edif_ensino
1,1,POINT (-44.04468 -19.99028),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.584961,node/1830091373_307923321060745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edif_ensino
2,2,POINT (-44.04470 -19.99031),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.592773,node/1830091373_982441749168567,11289.0,node_1830091373,edif_ensino,amenity_school,ses,s e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_ensino
3,3,POINT (-44.04463 -19.99021),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.440186,node/1830091373_798420737483679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edif_ensino
4,4,POINT (-44.04473 -19.99034),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.346680,node/1830091373_970234847060373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edif_ensino


In [15]:
# Manter apenas registros onde texto foi extraído
gdf_detected = gdf_merged_upd[gdf_merged_upd["path_crop"].notna()].copy()
display(gdf_detected)

,index_x,geometry,id_celula,classe_x,tag,value,@changesetId,@contributionChangesetId,@creation,@osmId,...,score,unique_key,index_y,osmId_formatado,classe_y,tag_value,words_sli,texto_sli,path_crop,classe
2,2,POINT (-44.04470 -19.99031),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.592773,node/1830091373_982441749168567,11289.0,node_1830091373,edif_ensino,amenity_school,ses,s e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_ensino
6,6,POINT (-44.04476 -19.99037),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.365479,node/1830091373_775809203082616,11287.0,node_1830091373,edif_ensino,amenity_school,plris,p l r i s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_ensino
9,9,POINT (-44.04462 -19.99025),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.407715,node/1830091373_354746259401102,11264.0,node_1830091373,edif_ensino,amenity_school,ie,i e,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_ensino
11,11,POINT (-44.04463 -19.99027),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.259277,node/1830091373_1074621596400304,11240.0,node_1830091373,edif_ensino,amenity_school,ettan,e t t a n,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_ensino
13,13,POINT (-44.04469 -19.99034),200ME60356N90866,edif_ensino,amenity,school,12291812,12291812.0,1,node/1830091373,...,0.396240,node/1830091373_2900608136921624,11255.0,node_1830091373,edif_ensino,amenity_school,es,e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_ensino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150273,29742,POINT (-43.98947 -19.84453),200ME60420N91026,cbge_praca,leisure,park,74217699,95159631.0,None,way/556003068,...,0.771484,way/556003068_223260119237378,20027.0,node_5363657452,edif_turistica,tourism_viewpoint,is,i s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,cbge_praca
150332,29801,POINT (-43.98921 -19.84463),200ME60420N91026,edif_turistica,tourism,viewpoint,55773766,55773766.0,1,node/5363657452,...,0.647461,node/5363657452_1143601826125378,20025.0,node_5363657452,edif_turistica,tourism_viewpoint,se,s e,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_turistica
150337,29806,POINT (-43.98889 -19.84489),200ME60420N91026,edif_turistica,tourism,viewpoint,55773766,55773766.0,1,node/5363657452,...,0.884277,node/5363657452_1183758152100706,20026.0,node_5363657452,edif_turistica,tourism_viewpoint,des,d e s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_turistica
150356,29825,POINT (-43.98947 -19.84453),200ME60420N91026,edif_turistica,tourism,viewpoint,55773766,55773766.0,1,node/5363657452,...,0.462891,node/5363657452_223260119237378,20027.0,node_5363657452,edif_turistica,tourism_viewpoint,is,i s,c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\results\6...,edif_turistica


In [ ]:
# Salvar geopackge (mpl_ocr_results_clip) só com detectados
# Caminho do GeoPackage de saída
base = os.getcwd()
out_dir = "results/6_YOLO_KerasOCR_results/mpl"

out_gpkg_detected = os.path.join(out_dir, "mpl_ocr_results_merged_clipMatches.gpkg")

gdf_detected.to_file(out_gpkg_detected, driver="GPKG")
print(f"[OK] GeoPackage 'detected' salvo em {out_gpkg_detected} com {len(gdf_detected)} registros")

[OK] GeoPackage 'detected' salvo em results/6_YOLO_KerasOCR_results/mpl\mpl_ocr_results_clip_detected.gpkg com 28526 registros
